# Trend Strategy for the EURUSD Exchange Rate

## Imports

We need to import some open source libraries for this analysis:

In [1]:
import pandas as pd  # for working with dataframes
import matplotlib.pyplot as plt # for visualisations

In [ ]:
sns.set_style('darkgrid') # 
